# Нейросети в задачих обработки текстов
**Разработчик: Алексей Озерин**

## Генерация коротких текстов с помощью RNN


Генерировать тексты можно с помощью RNN, предсказывающей следующий символ последовательности по предыдущим.

В этом задании предлагается написать и проучить на небольшом датасете имен [генеративную модель на основе символов -- Char-RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

![charseq](./charseq.jpeg)
Картинка взята из [статьи Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

В файле `names` находится ~8k имен на латинице.

Модель будет получать на вход имя `Amandy` и выдавать его же, только со сдвигом: `mandy `.

Чтобы сеть училась генерировать заглавные буквы, добавим в начало специальный токен, пробел:
```
_Amandy --> Amandy_
```

Для практического использования, на каждом шаге будем подавать на вход букву, предсказанную на предыдущем.
Так что нам потребуется правило для останова генерации (это может быть просто ограничение на количество шагов).
С другой стороны, можно добавить в конец каждого примера обучающей выборки специальный `<EOS>` токен. В данном случае обозначим его `#`:

```
_Amandy --> Amandy#
```

Будем прекращать генерацию при досрочном выпадании `<EOS>`.

In [ ]:
import os
start_token = " "
eos = '#'

with open("names") as f:
    names = f.readlines()
    names = [start_token + name.strip() + eos for name in names]

names = list(set(names))  # в датасете есть повторы
print('There are {} names: '.format(len(names)))
for x in names[::1000]:
    print(x)

In [ ]:
# TODO: постройте частоты употреблений букв
<your code>
# HINT: для графика возьмите plt.bar

In [ ]:
# датасете есть слова с разными длинами
MAX_LENGTH = max(map(len,names))
print("max length =", MAX_LENGTH)

plt.title('Sequence length distribution')
plt.hist(list(map(len,names)), bins=25);

In [ ]:
names[:10]

In [ ]:
# TODO: отберите уникальные токены и заполните два словаря для конвертации токенов <-> индексы
# сделайте так, чтобы пробел имел номер 0
    
tokens = <your code>
    
tok2id = <your code>
id2tok = <your code>

n_tokens = len(tokens)
print ('There are {} tokens',n_tokens)

assert 50 < n_tokens < 60

print('Vocabular: ' + "".join(tokens))

In [ ]:
def to_matrix(names, max_len=None, pad=tok2id[' '], dtype=np.int64):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, names))
    names_ix = np.zeros([len(names), max_len], dtype) + pad

    for i in range(len(names)):
        name_ix = list(map(tok2id.get, names[i]))
        names_ix[i, :len(name_ix)] = name_ix

    return names_ix

In [ ]:
print('\n'.join(names[:10]))
print(to_matrix(names[:10]))

In [ ]:
# TODO: разбейте все имена на тренировочную и тестовую часть
<your code>

train_data, val_data = split_data(names)

len(train_data), len(val_data)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
from IPython.display import clear_output

# Char-RNN для имен (0.2 балла)

Вам нужно написать сеть, кодирующую номера входных символов с помощью таблицы Embeddings. 
Получившиеся тензоры пропустить через RNN ячейку, затем преобразовать в логиты для предсказания номера нового символа.

In [ ]:
# NB: обратите внимание на порядок осей при вызове forward
# http://pytorch.org/docs/master/nn.html#recurrent-layers

# Сделайте возможность выбора типа ячейки, RNN, GRU или LSTM
# TODO: заполните пропуски. Функция forward будет вызываться на каждый шаг нами

class NameRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size, cell="rnn", n_layers=1):
        super(NameRNN, self).__init__()
        # добавьте возможность выбрать тип ячейки RNN/LSTM
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.cell = cell
        
        <your code>
        
    def forward(self, input, hidden):
        <your code>
        return output, hidden

    def init_hidden(self, batch_size):
        if self.cell == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

# Код для тренировки RNN (0.2 балла)

In [ ]:
def train_epoch(model, optimizer, train_batches):
    loss_log = []
    model.train()
    
    for batch in train_batches:
        # можно вынести подсчет ошибки в модельку
        
        nums = to_matrix(batch)
        <your code>
            
        loss = loss.data[0]
        loss_log.append(loss)
    return loss_log   

def test(model, test_batches):
    loss_log = []
    model.eval()
    for batch in test_batches:  
        
        nums = to_matrix(batch)
        <your code>
        
        loss = loss.data[0]
        loss_log.append(loss)
    return loss_log

def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)    
    points = np.array(val_history)
    plt.scatter(points[:, 0], points[:, 1], marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    
def train(model, opt, n_epochs):
    train_log = []
    val_log = []
    
    bs = 32
    total_steps = 0
    train_batches = np.array_split(train_data, len(train_data) // bs)
    test_batches = np.array_split(val_data, len(val_data) // bs)
    for epoch in range(n_epochs):
        train_loss = train_epoch(model, opt, train_batches)
        train_log.extend(train_loss)
        total_steps += len(train_batches)
        
        val_loss = test(model, test_batches)
        train_log.extend(train_loss)
        
        val_log.append((len(train_log), np.mean(val_loss)))
        
        clear_output()
        plot_history(train_log, val_log)

In [ ]:
rnn = NameRNN(len(tokens), 50, len(tokens), cell='rnn')

opt = torch.optim.Adam(rnn.parameters(), lr=1e-4)
train(rnn, opt, 20)

In [ ]:
rnn = NameRNN(len(tokens), 50, len(tokens), cell='lstm')

opt = torch.optim.Adam(rnn.parameters(), lr=1e-4)
train(rnn, opt, 20)

# Генерация по argmax (0.2 балла)

In [ ]:
# Напишите функцию генерации продолжения строки
def pick_by_argmax(logits):
    <your code>

def ids2string(ids):
    return "".join(id2tok[_] for _ in ids)


def gen_continuation(model, prefix=" "):
    hidden = model.init_hidden(1)
    nums = to_matrix(prefix)
    nums = Variable(torch.from_numpy(nums))
    
    # TODO: сначала сверните строку с помощью RNN:
    # нас интересует последний output и hidden
    <your code>
    
    # TODO: затем сгенерируйте несколько последующих символов
    # outs -- это массив с номерами токенов
    <your code>
    
    print(prefix + '|'+ ids2string(outs))
    
gen_continuation(rnn, " Ku")

# Генерация с семплированием (0.4 балла)

Обычный софтмакс 
$$p_i = \frac{\exp (x_i)}{\sum \exp (x_j)}$$
можно модернизировать с помощью температуры:
$$p_i = \frac{\exp (x_i / T)}{\sum \exp (x_j / T)}$$

Это позволит плавно переходить от выбора наиболее вероятного элемента ($T << 1$) до практически равновероятного ($T >> 1$)


In [ ]:
# Напишите функцию генерации батчами с семплированием из распределения и температурой
def batch2string(ids, prefix):
    # модифицируйте ids2string для работы с батчами
    <your code>

def pick_by_distribution(logits):
    # превратите логиты в распределение
    # затем семлируйте из него batch примеров
    <your code>


def gen_continuation_temp(model, prefix=" ", temperature=1.0, n=10):
    hidden = model.init_hidden(n)
    nums = to_matrix([prefix] * n)
    nums = Variable(torch.from_numpy(nums))

    # аналогично, сначала получите батч output, hidden
    <your code>
    
    # затем, сгенерируйте n последующих символов
    # в outs положите матрицу номеров токенов и отобразите ее
    
    print(batch2string(outs, prefix + '|'))

In [ ]:
gen_continuation_temp(rnn, prefix=" An", temperature=0.5, n=10)